In [1]:
import pandas as pd
import numpy as np
# import random as rn
import sklearn
# from scipy import stats
# import math
import re

from sklearn.preprocessing import  LabelEncoder #OneHotEncoder
# from sklearn import ensemble
from sklearn import linear_model

# from sklearn.model_selection import GridSearchCV,cross_val_score
# from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error,accuracy_score

# # from sklearn.ensemble import RandomForestClassifier
# from sklearn import linear_model

# import matplotlib.pyplot as plt
# import seaborn as sns
# from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet, RidgeCV, LassoCV

In [2]:
totDF = pd.read_csv('../../data/processed/Cleaned_Data_Set.csv')

## Cleaning / Sampling

In [3]:
r1 = re.compile('.*reporting')
r2 = re.compile('.*imputed')

cols_to_drop1 = list(filter((r1.match), totDF.columns))
cols_to_drop2 = list(filter((r2.match), totDF.columns))
cols_to_drop3 = ['admit_NICU']
cols_to_drop = cols_to_drop1 + cols_to_drop2 + cols_to_drop3

cols_to_keep = [col for col in totDF.columns if col not in cols_to_drop]

X_and_target = totDF[cols_to_keep + ['admit_NICU']].copy()

numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
catDF = X_and_target.select_dtypes(include=object).copy()
numDF = X_and_target.select_dtypes(include=numerics).copy() #only numeric columns

le = LabelEncoder()
catDF = catDF.apply(le.fit_transform)

concat_df = pd.concat([numDF,catDF],axis=1)

## GLM

In [4]:
sample_size_list = [100]

In [5]:
import warnings
warnings.filterwarnings('ignore')

In [6]:
#pure GLM

for sample_per_year in sample_size_list:
    dwnSmplDF = concat_df.groupby('birth_year',group_keys = False).apply(lambda x: x.sample(sample_per_year))
    
    cl_df = dwnSmplDF[cols_to_keep]
    encoded_target = dwnSmplDF['admit_NICU']
    
    glm = linear_model.LogisticRegression(solver = 'saga', 
                                              multi_class='auto', 
                                              n_jobs = -1,
                                              C = 1e4)
    print('sample size : %d\n' % (sample_per_year*5))
    %time glm.fit(cl_df, encoded_target)
    print('\nscore    : {0}'.format(glm.score(cl_df, encoded_target)))
    print('-'*50)
    

sample size : 500

CPU times: user 74.8 ms, sys: 12.5 ms, total: 87.3 ms
Wall time: 135 ms

score    : 0.934
--------------------------------------------------


In [ ]:
#GLM with Cross Validation
for sample_per_year in sample_size_list:
    dwnSmplDF = concat_df.groupby('birth_year',group_keys = False).apply(lambda x: x.sample(sample_per_year))
    
    cl_df = dwnSmplDF[cols_to_keep]
    encoded_target = dwnSmplDF['admit_NICU']
    
    glm_CV = linear_model.LogisticRegressionCV(#Cs = int(1e4),
                                                cv = 5,
                                                solver = 'saga',
                                                n_jobs = -1,
                                                random_state = 108
                                               ).fit(cl_df, encoded_target)
    print('sample size : %d\n' % (sample_per_year*5))
    %time glm_CV.fit(cl_df, encoded_target)
    print('\nscore    : {0}'.format(glm_CV.score(cl_df, encoded_target)))
    print('-'*50)


In [ ]:
'''
sample size : 500

CPU times: user 6min 47s, sys: 1min 26s, total: 8min 14s
Wall time: 1min 54s

score    : 0.932
--------------------------------------------------
'''

In [ ]:
#GLM with Lasso Penalty

for sample_per_year in sample_size_list:
    dwnSmplDF = concat_df.groupby('birth_year',group_keys = False).apply(lambda x: x.sample(sample_per_year))
    
    cl_df = dwnSmplDF[cols_to_keep]
    encoded_target = dwnSmplDF['admit_NICU']
    
    glm_lasso = linear_model.LogisticRegression(penalty = 'l1', 
                                              solver = 'saga', 
                                              multi_class='auto', 
                                              n_jobs = -1,
                                              C = 1e4)
    print('sample size : %d\n' % (sample_per_year*5))
    %time glm_lasso.fit(cl_df, encoded_target)
    print('\nscore    : {0}'.format(glm_lasso.score(cl_df, encoded_target)))
    print('-'*50)
    

In [7]:
#GLM with Lasso Penalty and Cross Validation

sample_size_list = [100,1000,10000]

for sample_per_year in sample_size_list:
    dwnSmplDF = concat_df.groupby('birth_year',group_keys = False).apply(lambda x: x.sample(sample_per_year))
    cl_df = dwnSmplDF[cols_to_keep]
    encoded_target = dwnSmplDF['admit_NICU']
    
    glm_lassoCV = linear_model.LogisticRegressionCV(#Cs = int(1e4),
                                                cv = 5,
                                                penalty = 'l1',
                                                solver = 'saga',
                                                n_jobs = -1,
                                                random_state = 108
                                               ).fit(cl_df, encoded_target)
    print('sample size : %d\n' % (sample_per_year*5))
    %time glm_lassoCV.fit(cl_df, encoded_target)
    print('\nscore    : {0}'.format(glm_lassoCV.score(cl_df, encoded_target)))
    print('-'*50)
    

sample size : 500

CPU times: user 2.87 s, sys: 97.3 ms, total: 2.96 s
Wall time: 571 ms

score    : 0.914
--------------------------------------------------
sample size : 5000

CPU times: user 2min, sys: 577 ms, total: 2min 1s
Wall time: 17.4 s

score    : 0.9308
--------------------------------------------------
sample size : 50000

CPU times: user 22min 12s, sys: 2.49 s, total: 22min 14s
Wall time: 3min 17s

score    : 0.9245
--------------------------------------------------


In [8]:
#GLM with Lasso Penalty and Cross Validation

sample_size_list = [20000]

for sample_per_year in sample_size_list:
    dwnSmplDF = concat_df.groupby('birth_year',group_keys = False).apply(lambda x: x.sample(sample_per_year))
    cl_df = dwnSmplDF[cols_to_keep]
    encoded_target = dwnSmplDF['admit_NICU']
    
    glm_lassoCV = linear_model.LogisticRegressionCV(#Cs = int(1e4),
                                                cv = 5,
                                                penalty = 'l1',
                                                solver = 'saga',
                                                n_jobs = -1,
                                                random_state = 108
                                               ).fit(cl_df, encoded_target)
    print('sample size : %d\n' % (sample_per_year*5))
    %time glm_lassoCV.fit(cl_df, encoded_target)
    print('\nscore    : {0}'.format(glm_lassoCV.score(cl_df, encoded_target)))
    print('-'*50)
    

sample size : 100000

CPU times: user 1h 8min 3s, sys: 13.1 s, total: 1h 8min 16s
Wall time: 5h 24min 52s

score    : 0.92539
--------------------------------------------------


In [10]:
glm_lassoCV.get_params

<bound method BaseEstimator.get_params of LogisticRegressionCV(Cs=10, class_weight=None, cv=5, dual=False,
                     fit_intercept=True, intercept_scaling=1.0, l1_ratios=None,
                     max_iter=100, multi_class='warn', n_jobs=-1, penalty='l1',
                     random_state=108, refit=True, scoring=None, solver='saga',
                     tol=0.0001, verbose=0)>

In [52]:
y_pred = glm_lassoCV.predict(cl_df)

In [54]:
from sklearn.metrics import confusion_matrix
print(confusion_matrix(encoded_target,y_pred))

[[89800     3   362]
 [  149  1047     2]
 [ 6945     0  1692]]


In [14]:
glmLCV_coefs = pd.DataFrame({'col' :list(cl_df.columns), 
                           'coef0': glm_lassoCV.coef_[0], 
                          'coef1': glm_lassoCV.coef_[1],
                          'coef2': glm_lassoCV.coef_[2]})

In [40]:
glmLCV_coefs

,col,coef0,coef1,coef2,abs_coef0,abs_coef1,abs_coef2
0,birth_year,-0.002558,-0.005061,0.003098,0.002558,0.005061,0.003098
1,birth_month,0.000523,-0.000090,-0.000224,0.000523,0.000090,0.000224
2,birth_time,-0.000149,0.000606,-0.000045,0.000149,0.000606,0.000045
3,birth_day_of_wk,0.000011,-0.000039,-0.000108,0.000011,0.000039,0.000108
4,birth_place,-0.001669,0.000396,-0.001239,0.001669,0.000396,0.001239
...,...,...,...,...,...,...,...
98,infant_living_at_report,-0.000385,-0.000048,0.000742,0.000385,0.000048,0.000742
99,infant_breastfed_at_discharge,0.001645,-0.000013,-0.001632,0.001645,0.000013,0.001632
100,cigs_before,0.001886,0.007419,-0.002703,0.001886,0.007419,0.002703
101,mothers_bmi_recode,-0.000629,0.000257,0.000834,0.000629,0.000257,0.000834


In [15]:
glmLCV_coefs['abs_coef0'] = glmLCV_coefs['coef0'].apply(abs)
glmLCV_coefs['abs_coef1'] = glmLCV_coefs['coef1'].apply(abs)
glmLCV_coefs['abs_coef2'] = glmLCV_coefs['coef2'].apply(abs)

In [45]:
top20_coef0 = glmLCV_coefs.nlargest(10,'abs_coef0')['col']
top20_coef1 = glmLCV_coefs.nlargest(10,'abs_coef1')['col']
top20_coef2 = glmLCV_coefs.nlargest(10,'abs_coef2')['col']

In [46]:
top20_coef0

77           obst_est_edit_wk
76      combined_gestation_wk
43               n_prev_cesar
60      final_delivery_method
71          APGAR_score_10min
79          assist_vent_immed
82    antibiotics_for_newborn
8                mothers_race
5                 mothers_age
80         assist_vent_after6
Name: col, dtype: object

In [47]:
top20_coef1

43               n_prev_cesar
100               cigs_before
25          n_prenatal_visits
24     mo_prenatal_care_began
0                  birth_year
102               weight_gain
21         prior_terminations
28                  cigs_tri2
8                mothers_race
29                  cigs_tri3
Name: col, dtype: object

In [48]:
top20_coef2

77           obst_est_edit_wk
76      combined_gestation_wk
71          APGAR_score_10min
60      final_delivery_method
79          assist_vent_immed
5                 mothers_age
82    antibiotics_for_newborn
43               n_prev_cesar
31                mothers_bmi
8                mothers_race
Name: col, dtype: object

In [49]:
len(list(set(top20_coef0).union(set(top20_coef1)).union(set(top20_coef2))))

19

In [50]:
list(set(top20_coef0).union(set(top20_coef1)).union(set(top20_coef2)))

['assist_vent_immed',
 'antibiotics_for_newborn',
 'birth_year',
 'assist_vent_after6',
 'mo_prenatal_care_began',
 'prior_terminations',
 'mothers_age',
 'final_delivery_method',
 'n_prev_cesar',
 'weight_gain',
 'obst_est_edit_wk',
 'cigs_tri2',
 'mothers_race',
 'cigs_before',
 'cigs_tri3',
 'combined_gestation_wk',
 'mothers_bmi',
 'n_prenatal_visits',
 'APGAR_score_10min']

In [51]:
list(set(top20_coef0).intersection(set(top20_coef1)).intersection(set(top20_coef2)))

['n_prev_cesar', 'mothers_race']

In [ ]:
'''
sample size : 100000

CPU times: user 54min 1s, sys: 37.3 s, total: 54min 39s
Wall time: 8min 39s

score    : 0.92512
--------------------------------------------------
'''

In [ ]:
#GLM with Lasso Penalty and Cross Validation

sample_size_list = [200000]

for sample_per_year in sample_size_list:
    dwnSmplDF = concat_df.groupby('birth_year',group_keys = False).apply(lambda x: x.sample(sample_per_year))
    cl_df = dwnSmplDF[cols_to_keep]
    encoded_target = dwnSmplDF['admit_NICU']
    
    glm_lassoCV = linear_model.LogisticRegressionCV(#Cs = int(1e4),
                                                cv = 5,
                                                penalty = 'l1',
                                                solver = 'saga',
                                                n_jobs = -2,
                                                random_state = 108
                                               ).fit(cl_df, encoded_target)
    print('sample size : %d\n' % (sample_per_year*5))
    %time glm_lassoCV.fit(cl_df, encoded_target)
    print('\nscore    : {0}'.format(glm_lassoCV.score(cl_df, encoded_target)))
    print('-'*50)
    

### RF

In [ ]:
#for sample_per_year in [100, 1000, 2000, 5000, 10000]:
for sample_per_year in [10000]:
    dwnSmplDF = concat_df.groupby('birth_year',group_keys = False).apply(lambda x: x.sample(sample_per_year))
    
    cl_df = dwnSmplDF[cols_to_keep]
    encoded_target = dwnSmplDF['admit_NICU']
    
    randomForest = ensemble.RandomForestClassifier(random_state = 108, warm_start = True)
    grid_para_forest = [{
        
        # fix the numbers as sample size increases
#         'n_estimators' : range(1000,10000,1000),
        'n_estimators' : np.linspace(10,int(np.sqrt(len(cl_df))),10,dtype=int),
#         'min_samples_split' : [100,10,2],
        'min_samples_leaf' : range(10,100,10)
        
    }]
    randomForest.set_params(random_state=108)
    grid_search_forest = GridSearchCV(randomForest, grid_para_forest, scoring='accuracy', cv=5, n_jobs=-1)
    %time grid_search_forest.fit(cl_df, encoded_target) #put in the df here
    print(
    '''Random Forest
sample size: {0}
best param : {1}
best score : {2}
r2         : {3}'''\
      .format(len(encoded_target), \
              grid_search_forest.best_params_,\
              grid_search_forest.best_score_, \
              r2_score(encoded_target, grid_search_forest.predict(cl_df)))
     )
    print('-'*60)

In [ ]:
sample_per_year = 10000
dwnSmplDF = concat_df.groupby('birth_year',group_keys = False).apply(lambda x: x.sample(sample_per_year))
cl_df = dwnSmplDF[cols_to_keep]
encoded_target = dwnSmplDF['admit_NICU']
best_params = grid_search_forest.best_params_
randomForest = ensemble.RandomForestClassifier(random_state = 108, warm_start = True, oob_score = True, **best_params)
randomForest.fit(cl_df, encoded_target)

In [ ]:
randomForest.oob_score_

In [ ]:
# list(randomForest.feature_importances_)

In [ ]:
data = {'col' :list(cl_df.columns), 'feature_importance': list(randomForest.feature_importances_)}
importance = pd.DataFrame(data)

In [ ]:
importance.nlargest(20,'feature_importance')

In [ ]:
list(importance.nlargest(30,'feature_importance')['col'])